In [1]:
!pip install eli5
import csv
import json

import numpy as np
import pandas as pd
# from datasets import load_dataset

from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc

# from transformers import Trainer, TrainingArguments
# from transformers import DataCollatorWithPadding, AutoTokenizer, AutoModelForSequenceClassification

# import torch

from tqdm import tqdm, trange
# from dataset_loader import load

from eli5 import show_weights
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# dataset_name = 'NarrativeQA'
# dataset = load(dataset_name, 'datasets/')
GOOGLE_PATH = "drive/MyDrive/si630-project/"
# all_df = pd.read_csv(GOOGLE_PATH + "RTQA-sample.csv")

In [4]:
# train_df  = pd.DataFrame.from_dict(dataset['train'])
# train_df.to_csv("squad_train.csv")
# dev_df = pd.DataFrame.from_dict(dataset['test'])
# dev_df.to_csv("squad_test.csv")
np.random.seed(42)
train_df = pd.read_csv(GOOGLE_PATH + "train_wiki.csv").sample(12000)
dev_df = pd.read_csv(GOOGLE_PATH + "test_wiki.csv").sample(3000)
train_df.shape, dev_df.shape

((12000, 2), (3000, 2))

In [5]:
train_df.head()

,text,label
202827,"Cobra Matata (also known as Banaloki Matata, J...",0
151507,Jonas Aukštuolis ( – 28 October 1949) was a Li...,0
163248,"In Etruscan mythology, Tuchulcha was a chthoni...",1
157083,Ontario Motor Speedway was a motorsport venue ...,1
69269,Great! Movies Action (stylized as GREAT! movie...,0


In [6]:
!pip install stanza
import stanza
from collections import defaultdict

def deprel_func(sentences, nlp):
    doc = nlp(sentences)
    deprel_dist = defaultdict(int)
    for sent in doc.sentences:
        for word in sent.words:
            deprel_dist[word.deprel] += 1
    return deprel_dist

nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [7]:
deprel_dist = [deprel_func(sent, nlp) for sent in tqdm(train_df['text'])]
keys = set().union(*(d.keys() for d in deprel_dist))

100%|██████████| 12000/12000 [45:02<00:00,  4.44it/s]


In [8]:
deprel_dist_dev = [deprel_func(sent, nlp) for sent in tqdm(dev_df['text'])]
dev_keys = set().union(*(d.keys() for d in deprel_dist))

100%|██████████| 3000/3000 [11:36<00:00,  4.31it/s]


In [9]:
all_keys = keys.union(dev_keys)

train_list = [{k: d.get(k, 0) for k in all_keys} for d in deprel_dist]
dev_list = [{k: d.get(k, 0) for k in all_keys} for d in deprel_dist_dev]

In [10]:
train_syn = pd.DataFrame(train_list)
train_mean = train_syn.mean(axis=0)
train_std = train_syn.std(axis=0)
dev_syn = pd.DataFrame(dev_list)

train_syn = (train_syn - train_mean) / train_std
dev_syn = (dev_syn - train_mean) / train_std

### Dummy Classifier

In [11]:
mf_dummy_clf = DummyClassifier(strategy="most_frequent")
mf_dummy_clf.fit(train_df.text, train_df.label)
mf_dummy_test_preds = mf_dummy_clf.predict(dev_df.text)
precision_recall_fscore_support(dev_df.label, mf_dummy_test_preds, average='binary')

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.0, 0.0, 0.0, None)

In [12]:
random_dummy_clf = DummyClassifier(strategy="stratified")
random_dummy_clf.fit(train_df.text, train_df.label)
random_dummy_test_preds = random_dummy_clf.predict(dev_df.text)
precision_recall_fscore_support(dev_df.label, random_dummy_test_preds, average='binary')

(0.49222447599729546, 0.49122807017543857, 0.491725768321513, None)

In [13]:
fpr, tpr, threshold = roc_curve(np.array(dev_df.label) , random_dummy_test_preds)
auc(fpr, tpr)

0.4982490812010263

### Sklearn

In [14]:
# Convert text to features using a simple tf-idf
# min_df = is the minimum occurrences for a word to a feature
vectorizer = TfidfVectorizer(min_df=10)
X_train = vectorizer.fit_transform(train_df.text.values.astype('U'))

# Train a Logisitic Regression classifier
clf = LogisticRegression()
clf.fit(X_train, train_df.label)

# Test the classifier by converting test data to features
X_test = vectorizer.transform(dev_df.text.values.astype('U'))
y_pred = clf.predict(X_test)
y_true = dev_df.label

# We'll using binary F1 to test whether we can detect sarcastic comments
precision_recall_fscore_support(y_true, y_pred, average='binary')


(0.8985115020297699, 0.8960863697705803, 0.8972972972972973, None)

In [15]:
fpr, tpr, threshold = roc_curve(np.array(y_true) , y_pred)
auc(fpr, tpr)

0.8986360702607843

In [16]:
pred_df = pd.DataFrame({'text': dev_df.text, 'label': dev_df.label, 'predict': y_pred})
pd.set_option('display.max_colwidth', None)
not_match_df = pred_df[pred_df.label != pred_df.predict]
not_match_df

,text,label,predict
14781,"Turów Power Station is a coal-fired power station in Bogatynia, Poland. The power station, operated by state-owned Polska Grupa Energetyczna via Oddział Elektrownia Turów, is fuelled by lignite extracted from the nearby Turów coal mine. Operations at the plant began in 1962. As of 2021 it supplied 5% of Poland's electricity and is the sole provider of heat and hot water to hospitals, schools and homes in Bogatynia. The plant initially consisted of ten 200 MW units, commissioned from 1962 to 1971. PGE undertook a US$1.6 billion modernization of units 1-6 of the plant in the early 1990s. Units 7-10 have been phased out. Unit 7 was retired in 2003. In 2010, Unit 8 was retired. Units 9 and 10 were decommissioned in 2012-2013. PGE has repowered Units 5 and 6 to co-incinerate biomass, and plans for co-firing of biomass in boilers 1, 2, 3, and 4. Units 1, 2, and 3 have been upgraded from 200 MW to 235 MW each. The plant's remaining six units have a combined capacity of 1,305 MW.",0,1
10664,"Jean-Guy is a given name. Notable people with the name include: In politics\nJean-Guy Allard (born 1948), Canadian journalist for Le Journal de Montréal and Le Journal de Québec\nJean-Guy Cardinal (1925–1979), nationalist politician in Quebec, Canada\nJean-Guy Carignan BA, MBA (born 1941), member of the Canadian House of Commons from 2000 to 2004\nJean-Guy Chrétien (born 1946), member of the Canadian House of Commons from 1993 to 2000\nJean-Guy Dagenais (born 1950), Canadian politician from Quebec\nJean-Guy Deschamps, former politician in Montreal, Quebec, Canada\nJean-Guy Dubé, Conservative Party of Canada candidate in the 2008 Canadian federal election\nJean-Guy Dubois (born 1948), Liberal party member of the Canadian House of Commons\nJean-Guy Guilbault (born 1931), member of the House of Commons of Canada\nJean-Guy Hudon (born 1941), Progressive Conservative member of the Canadian House of Commons\nJean-Guy Laforest (born 1944), business owner and former political figure in New Brunswick, Canada\nJean-Guy Péloquin, Abolitionist Party of Canada candidate in 1993 Canadian federal election\nJean-Guy Sabourin, Parti créditiste candidate in 1973 Quebec provincial election\nJean-Guy Trépanier, politician in the Quebec, Canada",0,1
56676,"In the year 1977, significant events in radio took place. The year saw the launch of the first FM radio station, WNUR in Newark, New Jersey. The year also saw the advent of satellite radio and the development of digital audio broadcasting.",1,0
25585,"William Brade (1560 – 26 February 1630) was an English poet.\n\nBrade was born in London, the son of a tailor. He was educated at Merchant Taylors' School and then at Brasenose College, Oxford, where he obtained his BA in 1581 and MA in 1585. He also studied law at the Inns of Court, but never practised.\n\nHe married Anne More, the daughter of Sir Thomas More, in 1590. They had four sons and four daughters.\n\nBrade's first published work was a translation of Lucan's ""Pharsalia"" (1592). He is chiefly remembered today for his poems, most notably ""The Shepheardes Calender"" (1579), ""The Garden"" (1585), and ""The Castle of Indolence"" (1616).",1,0
20307,"Harry Bache Smith (December 28, 1860 – January 1, 1936) was a writer, lyricist and composer. The most prolific of all American stage writers, he is said to have written over 300 librettos and more than 6000 lyrics. Some of his best-known works were librettos for the composers Victor Herbert and Reginald De Koven. He also wrote the book or lyrics for several versions of the Ziegfeld Follies. Smith was born in Buffalo, New York to Josiah Bailey Smith (born 1837) and Elizabeth Bach (born 1838). According to his autobiography First Nights and First Editions (Boston: Little, Brown, 1931), Smith's actual name at birth was Henry Bach Smith. He married twice. His first wife was Lena Reed (born August 21, 1868), whom he married on October 12, 1887 in Chicago, Illinois. They had a son named Sydney R

In [17]:
vocab = vectorizer.vocabulary_
id_to_vocab = {id_: word for word, id_ in vocab.items()}

show_weights(clf,  vec=vectorizer, feature_names=id_to_vocab, top=(10, 10))

Weight?,Feature
+8.083,is
+5.540,has
+5.485,including
+4.789,died
+4.690,served
+4.004,was
+3.946,also
+3.899,began
+3.381,became
+3.298,been


### Sklearn with Syn

In [18]:
vectorizer = TfidfVectorizer(min_df=10)
X_train = vectorizer.fit_transform(train_df.text.values.astype('U'))
X_train = np.concatenate((X_train.toarray(), train_syn.to_numpy()), axis=1)

# Train a Logisitic Regression classifier
clf = LogisticRegression()
clf.fit(X_train, train_df.label)

# Test the classifier by converting test data to features
X_test = vectorizer.transform(dev_df.text.values.astype('U'))
X_test = np.concatenate((X_test.toarray(), dev_syn.to_numpy()), axis=1)
y_pred = clf.predict(X_test)
y_true = dev_df.label

# We'll using binary F1 to test whether we can detect sarcastic comments
precision_recall_fscore_support(y_true, y_pred, average='binary')

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.9084022038567493, 0.8900134952766532, 0.8991138377641444, None)

In [19]:
fpr, tpr, threshold = roc_curve(np.array(y_true) , y_pred)
auc(fpr, tpr)

0.901199106004598

In [20]:
pred_df = pd.DataFrame({'text': dev_df.text, 'label': dev_df.label, 'predict': y_pred})
pd.set_option('display.max_colwidth', None)
not_match_df = pred_df[pred_df.label != pred_df.predict]
not_match_df

,text,label,predict
14781,"Turów Power Station is a coal-fired power station in Bogatynia, Poland. The power station, operated by state-owned Polska Grupa Energetyczna via Oddział Elektrownia Turów, is fuelled by lignite extracted from the nearby Turów coal mine. Operations at the plant began in 1962. As of 2021 it supplied 5% of Poland's electricity and is the sole provider of heat and hot water to hospitals, schools and homes in Bogatynia. The plant initially consisted of ten 200 MW units, commissioned from 1962 to 1971. PGE undertook a US$1.6 billion modernization of units 1-6 of the plant in the early 1990s. Units 7-10 have been phased out. Unit 7 was retired in 2003. In 2010, Unit 8 was retired. Units 9 and 10 were decommissioned in 2012-2013. PGE has repowered Units 5 and 6 to co-incinerate biomass, and plans for co-firing of biomass in boilers 1, 2, 3, and 4. Units 1, 2, and 3 have been upgraded from 200 MW to 235 MW each. The plant's remaining six units have a combined capacity of 1,305 MW.",0,1
55000,"The Rondo dwarf galago (Paragalago rondoensis) or Rondo bushbaby is a species of primate in the family Galagidae. The dwarf galagos are the smallest members of the genus Galagoides. It weighs less than 100 grams, making it the smallest known galago. It is endemic to Tanzania where its natural habitat is subtropical or tropical dry forests. It lives in an area reported in 2012 to be less than 100 square kilometers and is threatened by habitat loss due to logging. While it was discovered in the 1950s, the Rondo dwarf galago was deemed data deficient until 1996. In 1996, the Rondo bushbaby was fully described as a species. It is now listed as one of ""The World's 25 Most Endangered Primates."" In 2010, it was also added to the Zoological Society of London's list of genetically distinct and endangered mammals. Description\nBushbabies are small primates with long tails and large ears and eyes. They all have grooming claws, a tooth comb, and a pseudo-tongue. The Rondo bushbaby can be distinguished from other dwarf galagos by its bottle brush tail. The tail is red in young Rondo bushbabies, and darkens with age. It also has a distinctive ""double unit rolling call"". Two soft units comprise the call. The first is a higher pitch sound that can be repeated up to six times at a constant tempo. This forms a phrase.",0,1
28827,"Richard O'Neil Burrell Jr. (born September 24, 1971), better known by his stage name Robb Banks, is an American rapper and actor. He is the founder and head of the record label Robb Bank Records. Banks has released five studio albums: 1998's Robb Bank, 2002's The Great Depression, 2006's The Big Dirty, 2009's American Royalty, and 2013's The Money Store. Banks has also released three mixtapes: 2005's Burn One Down, 2007's God Bless America, and 2009's Gotta Be Real. Banks starred in the 2009 film Get Low alongside Robert Duvall and Sissy Spacek. \n\nBanks was born in Houston, Texas. He moved to Los Angeles at age 16 to pursue a career in rap music. Banks began his career as a member of the hip hop group World Class Wreckin' Cru, alongside artists such as MC Ren and Mac Mall. The group disbanded in 1994. In 1996, Banks released his debut solo album Robb Bank; it peaked at number 55 on the Billboard 200 chart. The following year, he released his second album The Great Depression; it peaked at number 21 on the Billboard 200 chart. Banks released his third album, 2006's The Big Dirty, which peaked at number 12 on the Billboard 200 chart. His fourth album, 2009's American Royalty, peaked at number six on the Billboard 200 chart. His fifth album, 2013's The Money Store, peaked at number one on the Billboard 200 chart. Banks has also released three mixtapes:",1,0
41952,"Coláiste Phádraig (St. Patrick's College) is a Gaelic-medium secondary school in Lucan, County Dublin, Ireland. It was founded in 1879.\n\nThe school is a member of the Irish National Education Council and the Irish Secondary Schools Athletic Associa

In [21]:
vocab = vectorizer.vocabulary_
id_to_vocab = {id_: word for word, id_ in vocab.items()}
for i, k in enumerate(all_keys, len(id_to_vocab)):
    id_to_vocab[i] = k

show_weights(clf,  vec=vectorizer, feature_names=id_to_vocab, top=(10, 10))

Weight?,Feature
+5.335,including
+4.625,also
+3.986,has
+3.023,served
+2.898,2006
+2.867,important
+2.652,most
+2.553,born
+2.485,began
+2.404,album


### Only Syn

In [22]:
vectorizer = TfidfVectorizer(min_df=10)
X_train = train_syn.to_numpy()

# Train a Logisitic Regression classifier
clf = LogisticRegression()
clf.fit(X_train, train_df.label)

# Test the classifier by converting test data to features
X_test = dev_syn.to_numpy()
y_pred = clf.predict(X_test)
y_true = dev_df.label

# We'll using binary F1 to test whether we can detect sarcastic comments
precision_recall_fscore_support(y_true, y_pred, average='binary')

(0.8342465753424657, 0.8218623481781376, 0.8280081577158396, None)

In [23]:
fpr, tpr, threshold = roc_curve(np.array(y_true) , y_pred)
auc(fpr, tpr)

0.8312210291615325

In [24]:
pred_df = pd.DataFrame({'text': dev_df.text, 'label': dev_df.label, 'predict': y_pred})
pd.set_option('display.max_colwidth', None)
not_match_df = pred_df[pred_df.label != pred_df.predict]
not_match_df

,text,label,predict
14781,"Turów Power Station is a coal-fired power station in Bogatynia, Poland. The power station, operated by state-owned Polska Grupa Energetyczna via Oddział Elektrownia Turów, is fuelled by lignite extracted from the nearby Turów coal mine. Operations at the plant began in 1962. As of 2021 it supplied 5% of Poland's electricity and is the sole provider of heat and hot water to hospitals, schools and homes in Bogatynia. The plant initially consisted of ten 200 MW units, commissioned from 1962 to 1971. PGE undertook a US$1.6 billion modernization of units 1-6 of the plant in the early 1990s. Units 7-10 have been phased out. Unit 7 was retired in 2003. In 2010, Unit 8 was retired. Units 9 and 10 were decommissioned in 2012-2013. PGE has repowered Units 5 and 6 to co-incinerate biomass, and plans for co-firing of biomass in boilers 1, 2, 3, and 4. Units 1, 2, and 3 have been upgraded from 200 MW to 235 MW each. The plant's remaining six units have a combined capacity of 1,305 MW.",0,1
55000,"The Rondo dwarf galago (Paragalago rondoensis) or Rondo bushbaby is a species of primate in the family Galagidae. The dwarf galagos are the smallest members of the genus Galagoides. It weighs less than 100 grams, making it the smallest known galago. It is endemic to Tanzania where its natural habitat is subtropical or tropical dry forests. It lives in an area reported in 2012 to be less than 100 square kilometers and is threatened by habitat loss due to logging. While it was discovered in the 1950s, the Rondo dwarf galago was deemed data deficient until 1996. In 1996, the Rondo bushbaby was fully described as a species. It is now listed as one of ""The World's 25 Most Endangered Primates."" In 2010, it was also added to the Zoological Society of London's list of genetically distinct and endangered mammals. Description\nBushbabies are small primates with long tails and large ears and eyes. They all have grooming claws, a tooth comb, and a pseudo-tongue. The Rondo bushbaby can be distinguished from other dwarf galagos by its bottle brush tail. The tail is red in young Rondo bushbabies, and darkens with age. It also has a distinctive ""double unit rolling call"". Two soft units comprise the call. The first is a higher pitch sound that can be repeated up to six times at a constant tempo. This forms a phrase.",0,1
11941,"Kitty Linn O'Neil (March 24, 1946 – November 2, 2018) was an American stuntwoman and racer, known as ""the fastest woman in the world"". An illness in early childhood left her deaf, and more illnesses in early adulthood cut short a career in diving. O'Neil's career as a stuntwoman and race driver led to her depiction in a television movie and as an action figure. Her women's absolute land speed record stood until 2019. Early life\nKitty Linn O'Neil was born in Corpus Christi, Texas on March 24, 1946. John O'Neil, her father, was an officer in the United States Army Air Forces, who had been an oil wildcatter. He died in an airplane crash during Kitty's childhood. Her mother, Patsy Compton O'Neil, was native Cherokee. At five months of age, O'Neil contracted simultaneous childhood diseases, losing her hearing. After her deafness became apparent at the age of two, her mother taught her lip-reading and speech, eventually becoming a speech therapist and co-founding a school for students with hearing impairment in Wichita Falls, Texas.",0,1
28827,"Richard O'Neil Burrell Jr. (born September 24, 1971), better known by his stage name Robb Banks, is an American rapper and actor. He is the founder and head of the record label Robb Bank Records. Banks has released five studio albums: 1998's Robb Bank, 2002's The Great Depression, 2006's The Big Dirty, 2009's American Royalty, and 2013's The Money Store. Banks has also released three mixtapes: 2005's Burn One Down, 2007's God Bless America, and 2009's Gotta Be Real. Banks starred in the 2009 film Get Low alongside Robert Duvall and Sissy Spacek. \n\nBanks was born in Hous

In [25]:
id_to_vocab = {}
for i, k in enumerate(all_keys):
    id_to_vocab[i] = k

show_weights(clf,  vec=vectorizer, feature_names=id_to_vocab, top=(10, 10))

Weight?,Feature
+1.079,root
+0.592,punct
+0.481,obl
+0.312,aux
+0.281,nmod
+0.224,cc
+0.211,mark
+0.196,nmod:poss
+0.188,obl:agent
+0.183,aux:pass
